In [ ]:
!pip install pretty_midi mir_eval numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.7/90.7 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.2 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592289 sha256=c5792fb09c46208ac62c6eba08e4b86905d26da520e696d151a574d72c7c08e9
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
  Created wheel for mir_eval: filename=mir_eval-0.7-py3-none-any.whl size=100704 sha256=8bd77c1cdb18c4923fff0ad7d72df2ad7a2640f11dab647a174d83d4dfc4fd62
  Stored in directory: /root/.cache/pip/wheels/3e/2f/0d/dda9c4c77a170e21356b6afa2f7d9bb078338634ba05d94e3f
Successfully built pretty_midi mir_eval
  Attempting uninstall: packa

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pretty_midi
import mir_eval
import numpy as np

os.chdir("/content/drive/MyDrive/Dissertation Code")

def read_note_data_from_text(file_path):
    intervals = []
    pitches = []
    with open(file_path, 'r') as file:
        for line in file:
            start, end, pitch = line.strip().split()
            intervals.append([float(start), float(end)])
            pitches.append(int(pitch))
    # print(np.array(intervals))
    return np.array(intervals), np.array(pitches)

# def read_note_data_from_text(file_path): # For OMAPS2
#     intervals = []
#     pitches = []
#     velocities = []
#     with open(file_path, 'r') as file:
#         for line in file:
#             columns = line.strip().split()
#             start, end, pitch, velocity = columns
#             intervals.append([float(start), float(end)])
#             pitches.append(int(pitch))
#             velocities.append(int(velocity))
#     return np.array(intervals), np.array(pitches)

# def prepare_data_for_evaluation(midi_file, cc_list=[64, 67], split_pedal=False):
#     midi_data = pretty_midi.PrettyMIDI(midi_file)
#     notes = []
#     for instrument in midi_data.instruments:
#         if instrument.program in range(0, 8):
#             notes.extend(instrument.notes)

#     intervals = []
#     pitches = []
#     velocities = []
#     for note in notes:
#         if note.pitch >= 0 or -note.pitch in cc_list:
#             intervals.append([note.start, note.end])
#             pitches.append(midi_to_freq(note.pitch))
#             velocities.append(note.velocity)

#     intervals = np.array(intervals)
#     pitches = np.array(pitches)
#     velocities = np.array(velocities)

#     return intervals, pitches, velocities

# def evaluate_midi(predicted_midi_file, truth_midi_file):
#     predicted_intervals, predicted_pitches, _ = prepare_data_for_evaluation(predicted_midi_file)
#     truth_intervals, truth_pitches, _ = prepare_data_for_evaluation(truth_midi_file)

#     precision, recall, f1_score, _ = mir_eval.transcription.precision_recall_f1_overlap(
#         truth_intervals, truth_pitches, predicted_intervals, predicted_pitches)

#     return precision, recall, f1_score

def prepare_data_for_evaluation(midi_file):
    midi_data = pretty_midi.PrettyMIDI(midi_file)
    intervals = []
    pitches = []
    for instrument in midi_data.instruments:
        for note in instrument.notes:
            intervals.append([note.start, note.end])
            pitches.append(note.pitch)
    # print(np.array(intervals))
    return np.array(intervals), np.array(pitches)

def read_note_data_from_npy(file_path):
    data = np.load(file_path)

    intervals = data[:, :2]
    pitches = data[:, 2]

    return intervals, pitches

def evaluate_midi(predicted_midi_file, truth_midi_file):
    predicted_intervals, predicted_pitches = read_note_data_from_npy(predicted_midi_file)
    truth_intervals, truth_pitches = read_note_data_from_text(truth_midi_file)

    print("done")

    # Full evaluation considering only onset
    full_precision, full_recall, full_f1_score, _ = mir_eval.transcription.precision_recall_f1_overlap(
        truth_intervals, truth_pitches, predicted_intervals, predicted_pitches,
        onset_tolerance=0.05, pitch_tolerance=50.0, offset_ratio=None, offset_min_tolerance=0.05, strict=False, beta=1.0)

    # Onset evaluation
    onset_precision, onset_recall, onset_f1_score = mir_eval.transcription.onset_precision_recall_f1(
        truth_intervals, predicted_intervals, onset_tolerance=0.05, strict=False, beta=1.0)

    # Offset evaluation
    offset_precision, offset_recall, offset_f1_score = mir_eval.transcription.offset_precision_recall_f1(
        truth_intervals, predicted_intervals, offset_ratio=0.2, offset_min_tolerance=0.05, strict=False, beta=1.0)

    return {
        'full': (full_precision, full_recall, full_f1_score),
        'onset': (onset_precision, onset_recall, onset_f1_score),
        'offset': (offset_precision, offset_recall, offset_f1_score)
    }

def evaluate_folders(predicted_folder, truth_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    full_eval_path = os.path.join(output_folder, "full_evaluation_results.txt")
    onset_eval_path = os.path.join(output_folder, "onset_evaluation_results.txt")
    offset_eval_path = os.path.join(output_folder, "offset_evaluation_results.txt")
    full_avg_path = os.path.join(output_folder, "full_evaluation_average.txt")
    onset_avg_path = os.path.join(output_folder, "onset_evaluation_average.txt")
    offset_avg_path = os.path.join(output_folder, "offset_evaluation_average.txt")

    full_scores = []
    onset_scores = []
    offset_scores = []

    # Open the files outside the loop to write headers
    with open(full_eval_path, 'w') as full_file, open(onset_eval_path, 'w') as onset_file, open(offset_eval_path, 'w') as offset_file:
        full_file.write("File, Precision, Recall, F1 Score\n")
        onset_file.write("File, Precision, Recall, F1 Score\n")
        offset_file.write("File, Precision, Recall, F1 Score\n")

    for predicted_file in os.listdir(predicted_folder):
        if not predicted_file.endswith('.npy'):
            # print(predicted_file)
            continue  # Skip files that are not MIDI files

        if predicted_file.endswith('-transcription.npy'):
            # print(predicted_file)
            predicted_path = os.path.join(predicted_folder, predicted_file)
            truth_file = predicted_file.replace('-transcription.npy', '.txt')
            truth_path = os.path.join(truth_folder, truth_file)

        # print(truth_path, predicted_path)

        if os.path.isfile(truth_path):
            try:
                results = evaluate_midi(predicted_path, truth_path)

                # Append scores for averages calculation
                full_scores.append(results['full'])
                onset_scores.append(results['onset'])
                offset_scores.append(results['offset'])

                # Write detailed evaluation results
                with open(full_eval_path, 'a') as full_file, open(onset_eval_path, 'a') as onset_file, open(offset_eval_path, 'a') as offset_file:
                    full_file.write(f"{predicted_file}, {results['full'][0]}, {results['full'][1]}, {results['full'][2]}\n")
                    onset_file.write(f"{predicted_file}, {results['onset'][0]}, {results['onset'][1]}, {results['onset'][2]}\n")
                    offset_file.write(f"{predicted_file}, {results['offset'][0]}, {results['offset'][1]}, {results['offset'][2]}\n")

            except ValueError as e:
                print("Error")
                pass

    def calculate_and_save_averages(scores, file_path):
        if scores:  # Ensure there are scores to calculate averages
            avg_precision = sum(score[0] for score in scores) / len(scores)
            avg_recall = sum(score[1] for score in scores) / len(scores)
            avg_f1 = sum(score[2] for score in scores) / len(scores)
            with open(file_path, 'w') as file:
                file.write(f"Average Precision, Average Recall, Average F1 Score\n")
                file.write(f"{avg_precision}, {avg_recall}, {avg_f1}\n")

    calculate_and_save_averages(full_scores, full_avg_path)
    calculate_and_save_averages(onset_scores, onset_avg_path)
    calculate_and_save_averages(offset_scores, offset_avg_path)

# Usage example
predicted_folder = "OMAPS/evaluation/attack-delay"
truth_folder = "OMAPS/complete/text"

evaluate_folders(predicted_folder, truth_folder, predicted_folder)


done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done


In [ ]:
predicted_intervals, predicted_pitches = read_note_data_from_npy("OMAPS2/evaluation/attack-delay/01_01-transcription.npy")
truth_intervals, truth_pitches = read_note_data_from_text("OMAPS2/complete/text/01_01.txt")

print("Predicted Intervals:", predicted_intervals)
print("Predicted Pitches:", predicted_pitches)
print("Truth Intervals:", truth_intervals)
print("Truth Pitches:", truth_pitches)

FileNotFoundError: [Errno 2] No such file or directory: 'OMAPS2/evaluation/attack-delay/01_01-transcription.npy'